## Import Modules

In [1]:
import dataset as ds
import config as cfg
import tools
seed_time = tools.generate_seed()

import torch
from ultralytics import settings
from ultralytics import YOLO

from clearml import Task
import clearml
clearml.browser_login()
import gc


# ClearML
https://app.clear.ml/dashboard

# Yolo Config

In [2]:
# Ultralytics settings
settings.reset()
settings.update({'datasets_dir': cfg.YOLO_DATASET_DIR.replace('/','\\')})
print(settings)

# GPU Utilisation
print(torch.cuda.is_available())
device = torch.device("cuda")
print(torch.backends.cudnn.version())


{'settings_version': '0.0.4', 'datasets_dir': 'C:\\github\\Third-Year-Project\\Intruder-Aircraft-Detection\\datasets\\YOLOv8', 'weights_dir': 'weights', 'runs_dir': 'runs', 'uuid': 'fdb5c10788ffaa41a9047dc764dd8a0a3287d6bec12c0c66234ac745c0366efa', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}
True
8801


## Load Dataset

In [3]:
# Base paths for the images and labels
train_images_path = f'{cfg.BASE_DATASET}/images/train'
train_labels_path = f'{cfg.BASE_DATASET}/labels/train'
val_images_path = f'{cfg.BASE_DATASET}/images/valid'
val_labels_path = f'{cfg.BASE_DATASET}/labels/valid'

# Base path for metadata
metadata_path = f'{cfg.BASE_DATASET}/metadata.json'

# Create the DataFrames for the train and validation sets
train_df = ds.create_dataframe(train_images_path, train_labels_path, metadata_path)
valid_df = ds.create_dataframe(val_images_path, val_labels_path, metadata_path)

#train_df.info()

In [4]:
# augmentation metadata

#augmentation_metadata = {
#    'methods': {
#        'hsv': {
#            'parameters': {
#                'type': 'h',  # Probability of applying the augmentation
#                'shift': 50,
#                'p': 1.0
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'flip': {
#            'parameters': {
#                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'rotate': {
#            'parameters': {
#                'angle': 90,  # Rotation angle
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 30% of the training images
#        },
#        'bnc': {
#            'parameters': {
#                'alpha': 1.5,  # contrast control
#                'beta': 50, # brightness control
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'histEq': {
#            'parameters': {
#                'p': 1.0  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'whiteBal': {
#            'parameters': {
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'sharpen': {
#            'parameters': {
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'gaussian': {
#            'parameters': {
#                'var_limit': (10.0, 50.0) # range of noise varaince
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        },
#        'zoom': {
#            'parameters': {
#                'zoom_factor': 3  # Probability of applying the augmentation
#            },
#            'apply_to_percentage': 0.1  # 50% of the training images
#        }       
#    }
#}


# Test Parameters

In [5]:
# name dataset to be create as well as task name
dataset_name = 'test250'

# project and task constant parameters
#project_name= cfg.YOLO_PROJECT_NAME
dataset_dir = f'{cfg.YOLO_DATASET_DIR}/{dataset_name}'
project_dir = f'{cfg.YOLO_PROJECT_DIR}/{dataset_name}/'
class_names = cfg.YOLO_CLASS_NAMES

# task specifc training parameters
epochs = 10
model_variant = "yolov8n"
RUN = 1

task_name= task_name = 'auto-test8'
task_name = f'{task_name}-{epochs}-{RUN}'
project_name= cfg.YOLO_PROJECT_NAME + f'/automation-test'

"""
Methods: 
 - flip
 - rotate
 - bnc
 - gaussian
 - histEq
 - whiteBal
 - sharpen
 - zoom
 - hsv
"""

augmentation_metadata = {
    'methods': {        
        'flip': {
            'parameters': {
                'orientation': 'h',  # Could be 'h' for horizontal or 'v' for vertical
                'p': 1.0  # Probability of applying the augmentation
            },
            'apply_to_percentage': 0.5  # 50% of the training images
        }        
    }
}

train_size = 250
val_size = int(train_size * 0.2)

# Create Datasets

In [6]:
from sklearn.model_selection import train_test_split

_, test_train_df = train_test_split(
    train_df,
    test_size=train_size,  # Number of items you want in your sample
    stratify=train_df['ac'],  # Stratify based on the combined column
    random_state=seed_time  # Ensures reproducibility
)

_, test_val_df = train_test_split(
    valid_df,
    test_size=val_size,  # Number of items you want in your sample
    stratify=valid_df['ac'],  # Stratify based on the combined column
    random_state=seed_time  # Ensures reproducibility
)

# create sub dataset
ds.create_sub_dataset(dataset_dir, test_train_df, test_val_df, class_names)

# augment dataset
ds.augment_dataset(dataset_dir, augmentation_metadata)

Removing dataset if pre-existing
Copying training files:


Copying files: 100%|██████████| 500/500 [00:00<00:00, 1268.67it/s]


Copying validation files:


Copying files: 100%|██████████| 100/100 [00:00<00:00, 1652.10it/s]


Dataset 'test250' created at C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test250


Applying augmentations: 100%|██████████| 125/125 [00:01<00:00, 98.40it/s] 

An error occurred: Expected y_min for bbox (0.423786, -0.1070265, 1.094306, 0.33998649999999997, 0) to be in the range [0.0, 1.0], got -0.1070265.


# Training

In [7]:
# train on pure dataset

# dataset location
dataset_path=f'{dataset_dir}\\{dataset_name}.yaml'
project =  project_dir + 'pure' #save_dir # weight save path

# Create ClearML task
task = Task.init(
    project_name=project_name,
    task_name=f"{task_name}-pure"
)
task.set_parameter("model_variant", model_variant)

# Define Yolo model
#model = YOLO(f'{model_variant}.yaml') # train on model which is not pre-trained
model = YOLO(f'{model_variant}.pt')

#train args
args = dict(data=dataset_path, 
            epochs=epochs, 
            device=0, 
            #close_mosaic=epochs, # disable mosaic augmentation
            seed=42
            )
task.connect(args)

# train model
results=model.train(**args, project=project)

# validate model
metrics = model.val()
metrics_dict = {
    'mAP_50-95': metrics.box.map,     # Mean Average Precision from IoU=0.50 to 0.95
    'mAP_50': metrics.box.map50,      # Mean Average Precision at IoU=0.50
    'mAP_75': metrics.box.map75,      # Mean Average Precision at IoU=0.75
    'mAP_per_class': metrics.box.maps, # List of mAP from IoU=0.50 to 0.95 for each category
    'Precision': metrics.box.mp,      # Precision
    'Recall': metrics.box.mr          # Recall
}

task.upload_artifact('VAL_METRICS', tools.pretty_print_dict(metrics_dict))

# close task for next run
task.close()

#clear variables from memory
task = None
model = None
args = None
results = None


ClearML Task: created new task id=ad6d9fa10a59403f9753224736406b19
2024-05-07 02:29:39,106 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/e51413769c444f7788933dc4df4fad17/experiments/ad6d9fa10a59403f9753224736406b19/output/log
2024-05-07 02:29:44,108 - clearml.model - INFO - Selected model id: 6932d54ba60d49f7bd3432493468f8c1
New https://pypi.org/project/ultralytics/8.2.10 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.8  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets/YOLOv8/test250\test250.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/logs/YOLOv8/test250/pure, name=train4, exist_ok=False, pr

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test250\labels\train... 250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 250/250 [00:00<00:00, 1587.09it/s]


train: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test250\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test250\labels\valid... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<00:00, 473.80it/s]

val: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test250\labels\valid.cache


...


10 epochs completed in 0.019 hours.
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\pure\train4\weights\last.pt, 6.2MB
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\pure\train4\weights\best.pt, 6.2MB

Validating C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\pure\train4\weights\best.pt...
Ultralytics YOLOv8.2.8  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.11it/s]


                   all         50         50      0.608       0.46      0.463      0.206
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\pure\train4
2024-05-07 02:31:54,291 - clearml.storage - INFO - Starting upload: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\pure\train4\weights\best.pt => https://files.clear.ml/Intruder-Aircraft-Detection/YOLOv8/automation-test/auto-test8-10-1-pure.ad6d9fa10a59403f9753224736406b19/models/best.pt
Ultralytics YOLOv8.2.8  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test250\labels\valid.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.93s/it]


2024-05-07 02:32:03,573 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Intruder-Aircraft-Detection/YOLOv8/automation-test/auto-test8-10-1-pure.ad6d9fa10a59403f9753224736406b19/models/best.pt
                   all         50         50      0.622       0.46      0.464      0.206
Speed: 0.8ms preprocess, 13.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\pure\train42


In [8]:
# train on augmented dataset

# clear variables
torch.cuda.empty_cache()
gc.collect()

# dataset location
dataset_path=f'{dataset_dir}\\{dataset_name}-aug.yaml'
project =  project_dir + 'augmented' #save_dir # weight save path

# Create ClearML task
task = Task.init(
    project_name=project_name,
    task_name=f"{task_name}-aug"
)
task.set_parameter("model_variant", model_variant) 

# Define Yolo model
#model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

# train model
results = model.train(data=dataset_path, 
                      epochs=epochs, 
                      device=0, 
                      project=project,
                      #close_mosaic=epochs, # disable mosaic augmentation
                      seed=42)

# validate model
metrics = model.val()
metrics_dict = {
    'mAP_50-95': metrics.box.map,     # Mean Average Precision from IoU=0.50 to 0.95
    'mAP_50': metrics.box.map50,      # Mean Average Precision at IoU=0.50
    'mAP_75': metrics.box.map75,      # Mean Average Precision at IoU=0.75
    'mAP_per_class': metrics.box.maps, # List of mAP from IoU=0.50 to 0.95 for each category
    'Precision': metrics.box.mp,      # Precision
    'Recall': metrics.box.mr          # Recall
}

task.upload_artifact('VAL_METRICS', tools.pretty_print_dict(metrics_dict))
task.upload_artifact('AUGMENTATION_METADATA', tools.pretty_print_dict(augmentation_metadata))


# close task for next/last run
task.close()



ClearML Task: created new task id=6bcb47b99d3c4aebbea63be90c039622
ClearML results page: https://app.clear.ml/projects/e51413769c444f7788933dc4df4fad17/experiments/6bcb47b99d3c4aebbea63be90c039622/output/log


Connecting multiple input models with the same name: `yolov8n`. This might result in the wrong model being used when executing remotely


New https://pypi.org/project/ultralytics/8.2.10 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.8  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/datasets/YOLOv8/test250\test250-aug.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=C:/github/Third-Year-Project/Intruder-Aircraft-Detection/logs/YOLOv8/test250/augmented, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True,

train: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test250\labels\train-aug... 374 images, 0 backgrounds, 0 corrupt: 100%|██████████| 374/374 [00:00<00:00, 822.70it/s]

train: New cache created: C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test250\labels\train-aug.cache


...


10 epochs completed in 0.023 hours.
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\augmented\train3\weights\last.pt, 6.2MB
Optimizer stripped from C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\augmented\train3\weights\best.pt, 6.2MB

Validating C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\augmented\train3\weights\best.pt...
Ultralytics YOLOv8.2.8  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.96it/s]


                   all         50         50      0.634      0.519      0.481       0.24
Speed: 0.1ms preprocess, 1.0ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\augmented\train3
Ultralytics YOLOv8.2.8  Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 16379MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients


val: Scanning C:\github\Third-Year-Project\Intruder-Aircraft-Detection\datasets\YOLOv8\test250\labels\valid.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:08<00:00,  2.05s/it]


                   all         50         50       0.61        0.5      0.472      0.237
Speed: 0.9ms preprocess, 17.4ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to C:\github\Third-Year-Project\Intruder-Aircraft-Detection\logs\YOLOv8\test250\augmented\train32
